In [1]:
import wandb
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
from model_fusion.config import BASE_DATA_DIR, CHECKPOINT_DIR
from model_fusion.datasets import DataModuleType
from model_fusion.models import ModelType
from model_fusion.models.lightning import BaseModel 
from Experiments import lmc_experiment
from Experiments import baselines_experiment
from Experiments import otfusion_experiment
import pyhessian as hessian
from Experiments import pyhessian_experiment

# set seed for numpy based calculations
NUMPY_SEED = 100
np.random.seed(NUMPY_SEED)

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
c:\Users\filos\OneDrive\Desk

In [2]:
print("------- Loading models -------")

# select wandb run names
runA = '95shkeqb'
runB = '95shkeqb'

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runA}')

batch_size = run.config['batch_size']

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = {'batch_size': batch_size, 'data_dir': BASE_DATA_DIR}

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

model_hparams = list(filter(lambda x: 'model_hparams/' in x[0], run.config.items()))
model_hparams = {k.split('/')[1]: v for k, v in model_hparams}

checkpointA = f'model-fusion/Model Fusion/model-{runA}:best'
checkpointB = f'model-fusion/Model Fusion/model-{runB}:best'

run = wandb.init()

artifact = run.use_artifact(checkpointA, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelA = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

artifact = run.use_artifact(checkpointB, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelB = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

------- Loading models -------


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: frambelli (model-fusion). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-95shkeqb:best, 85.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4
wandb: Downloading large artifact model-95shkeqb:best, 85.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


In [ ]:
# LMC barrier
print("------- Computing LMC barrier -------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA,
    modelB=modelB,
)

In [ ]:
# Baselines (prediction ensembling, vanilla averaging)
print("------- Computing baselines -------")

wandb_tag = f'baselines-{runA}-{runB}'

vanilla_averaging_model = baselines_experiment.run_baselines(
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
)

In [ ]:
# OT model fusion + eval aligned model 
print("------- Computing model fusion -------")

wandb_tag = f"ot_model_fusion-{runA}-{runB}"

ofused_model, aligned_base_model = otfusion_experiment.run_otfusion(
    batch_size=batch_size,
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag
)

In [3]:
# Pyhessian I (compute sharpness and eigenspectrum of base models, vanilla avg and ot fusion solutions)
print("------- Computing Pyhessian I -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA)

top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()
print("The top Hessian eigenvalue of this model is %.4f"%top_eigenvalues[-1])






------- Computing Pyhessian I -------
Files already downloaded and verified
Files already downloaded and verified


ValueError: too many values to unpack (expected 2)

In [ ]:
# finetuning


In [ ]:
# Comparison II (compute sharpness of finetuned solutions)